In [ ]:
import json
import pyspark.sql.functions as F 
file="dz-faw-vw-vehicle-gateway-start.sh.log-20190213.gz.json"
df=sqlContext.read.json('file:///data/json/'+file)
schema=df.schema
df.printSchema()

In [ ]:
x=schema['data'].json()
import json
x=json.loads(x)
names=[y['name'] for y in x['type']['fields']]


In [ ]:
from pyspark.sql.types import FloatType,IntegerType,StringType   #raw is dictionary 
for i in names:
    if i in raw:
        l='data.'+i+'.val'
        df=df.withColumn(d[i],df[l].cast(FloatType()))

df=df.drop('data')

In [ ]:
df=df.filter(df.Temperature_difference_warning.isNotNull())

#convert unixtime to normal timestamp
from pyspark.sql.types import FloatType
df=df.withColumn('tm',(df['pkgTs']/1000))
df=df.withColumn('tm',F.from_unixtime('tm'))
df=df.withColumn('tm',F.from_utc_timestamp('tm','Asia/Chongqing'))


In [ ]:
#create trip column, 15 mins gap is defined as new trip
from pyspark.sql.window import Window
import sys
window=Window.partitionBy(df['vin']).orderBy(df['tm'].asc())
df=df.withColumn('tm_lag',F.lag(df['pkgTs']).over(window))
df=df.withColumn('gap',(df['pkgTs']-df['tm_lag'])/1000)

df=df.withColumn('new_trip',F.when((df['gap']>=900)|(F.isnull(df['gap'])),1).otherwise(0))
df=df.withColumn('trip_name',F.when(df['new_trip']==1,F.concat(F.col('vin'),F.lit('_'),F.col('pkgTs'))))
df=df.withColumn("trip_name", F.last('trip_name', True).over(window.rowsBetween(-sys.maxsize, 0)))

In [ ]:
#get alarm
alarm_cols=['Temperature_difference_warning', 'Battery_high_temperature_warning', 'Vehicle_energy_storage_device_type_over_voltage_warning', 'Vehicle_energy_storage_device_type_under_voltage_warning', 'Low_SOC_warning', 'Cell_over_voltage_warning', 'Cell_under_voltage_warning', 'Excessively_high_SOC_warning', 'SOC_jump_warning', 'Chargeable_energy_storage_system_unmatched_warning', 'Cell_poor_consistency_warning', 'Insulation_warning', 'DC_DC_temperature_warning', 'Brake_system_warning', 'DC_DC_state_warning', 'Electrical_machine_controller_temperature_warning', 'High_voltage_interlocking_state_warning', 'Electrical_machine_temperature_warning']

df=df.withColumn( "alarms",F.concat(*[F.when(F.col(c)>0,F.concat(F.lit(c),F.lit(","))).otherwise('') for c in alarm_cols])) 
df=df.withColumn("alarms",F.regexp_replace("alarms",".$",""))

df=df.withColumn('new_alarm',F.lag(df['alarms']).over(window))
df=df.withColumn('new_alarm',F.when(F.col('alarms')=='',0).when(F.col('new_alarm').isNull(),1).when(F.col('alarms')!=F.col('new_alarm'),1).otherwise(0))

In [ ]:
#alarm aggregtion and summary by vin and trip
df_explode=df.withColumn('alarm',F.explode(F.split('alarms',',')))
df1=df_explode.filter(df_explode['Highest_warning_level']>0)
df1=df1.groupby('vin','trip_name','alarms','Highest_warning_level').agg(F.min(df1.tm).alias('start_time'),F.max(df1.tm).alias('end_time'),F.sum(df1.new_alarm).alias('times'))
df1.show()

In [ ]:
from pyspark.sql.functions import udf

x=list('abcdefghijklmnopqrstuvwxyz'.upper())
y=list(range(2010,2036))
d_year=dict(zip(x,y))
udf1=udf(lambda x:d_year[x[9:10]],StringType())
df1=df1.withColumn('model_yr',udf1(df1['vin']))

m=['G1','GA','4G','4X','48','4A','GE','4L','4M']
n=['Q2','Q2','C7','A6/C7','A6_CKD','C8','e-tron','Q7','Q7']
d_model=dict(zip(m,n))
udf2=udf(lambda x:d_model[x[6:8]],StringType())
df1=df1.withColumn('model',udf2(df1['vin']))

df1=df1.withColumn('type',F.when(df1['vin'].substr(0,1)=='L','CKD').otherwise('FBU'))

In [ ]:
cond = [df.vin==df1.vin,df.tm > df1.start_time- F.expr('INTERVAL 15 MINUTES'), df.tm <df1.end_time+F.expr('INTERVAL 15 MINUTES')]
df2=df.join(df1, cond, 'inner').select(df.vin,df.Highest_warning_level,df.Vehicle_energy_storage_device_type_over_charging,df.Charging_state,df.DC_DC_state,df.Insulation_resistance,df.Total_voltage,df.Total_current,df.SOC,df.Fuel_consumption,df.Crankshaft_speed,df.ID_No__of_battery_subsystem_with_max__voltage,df.ID_No__of_cell_with_max__voltage,df.Max__cell_voltage,df.ID_No__of_battery_subsystem_with_min__voltage,df.ID_No__of_cell_with_min__voltage,df.Min__cell_voltage,df.ID_No__of_subsystem_with_max__temperature,df.Code_of_single_probe_with_max__temperature,df.Max__temperature_value,df.ID_No__of_subsystem_with_min__temperature,df.Code_of_probe_subsystem_with_min__temperature,df.Min__temperature_value,df.Longitude,df.Latitude,df.Engine_state,df.Accumulated_mileage,df.Gear_position,df.Vehicle_speed,df.Vehicle_state,df.Operation_mode,df.Stroke_of_accelerator_pedal,df.Brake_pedal_state,df.tm,df.trip_name,df.alarms)

In [ ]:
# read to jdbc
properties = {
    "user": "xinzhu",
    "password": "Itp_audi_2019",
    
    "drive":"org.postgresql.Driver"
}
df1.write.jdbc(url="jdbc:postgresql://192.168.22.179/audi",table='alarm_summary',properties=properties,mode='append')

# read from jdbc 
query="(select distinct(date(start_time)) as d from alarm_summary order by d asc) f"
df_db= sqlContext.read.format('jdbc')\
    .option("url", "jdbc:postgresql://192.168.22.179/audi") \
    .option("dbtable",query)\
    .option("user", "xinzhu") \
    .option("password", "Itp_audi_2019") \
    .option("driver","org.postgresql.Driver") \
    .load()


In [ ]:
# E015 is cell voltage, 128 cells. each has sts and val 
from pyspark.sql.functions import col, size
df.select(size(df['data.E015.val.val'])).limit(3).show()

In [ ]:
df=df.withColumn('v',df['data.E015.val.val'])  #return a list of 128 values

In [ ]:
from pyspark.sql.types import ArrayType,FloatType,IntegerType,StringType
from pyspark.sql.functions import udf
def c(x):
    t=[float(i) for i in x]
    a= sum(t)/len(t)
    return (sum([(float(i)-a)**2 for i in x])/len(x))**0.5
std_= udf(c,FloatType())
df.withColumn('v_std',std_(df['data.E015.val.val'])).limit(3).show()